# KNeighborsClassifier

Dieses Notebook beschäftigt sich mit dem Klassifikator, der das Votum für 'k-nearest neighbors' implementiert. Angewandt wird dieser Algorithmus auf den MNIST Datensatz, welcher in Summer aus 70.000 handgeschriebenen Ziffern von 0 bis 9 besteht. Diese Bilder sind jeweils in einem 28x28 Pixelformat hinterlegt.

Der Datensatz wird über die Website vom Prof. Yann LeCun bereitgestellt. Zusätzlich findet ihr dort Anweisungen, wie sein custom-file-format 'ubyte' gelesen werden kann.
http://yann.lecun.com/exdb/mnist/

Der Datensatz wird gesplittet in 60.000 Trainingsdaten und 10.000 Testdaten.

## Lesen des MNIST Datensatzes

In [3]:
import numpy as np
import requests
import gzip
import os
import hashlib

# Fetch data function (Function to read/write external data like MNIST dataset)
def fetch(file_name):
  if os.path.isfile(file_name):
    with open(file_name, "rb") as f:
      dat = f.read()
  else:
    with open(file_name, "wb") as f:
      dat = requests.get(file_name).content
      f.write(dat)
  return np.frombuffer(gzip.decompress(dat), dtype=np.uint8).copy()

In [4]:
# Read MNIST Dataset
# MNIST dataset consists of four files
mnist_files = {
	0 : "train-images-idx3-ubyte.gz",	# large train set images  	60.000 img
	1 : "train-labels-idx1-ubyte.gz",	# large label set 			60.008 lbl
	2 : "t10k-images-idx3-ubyte.gz",	# small train set images	10.000 img
	3 : "t10k-labels-idx1-ubyte.gz"		# small label set			10.008 lbl
	}

# All data will be written in X_and_Y in first place
X_and_y = []

# Reading MNIST dataset
for i, file in mnist_files.items():
	if i == 0 or i == 2:
		X_and_y.append(i)
		# [0x10:] 5th bit set
		# https://stackoverflow.com/questions/16405354/python-confused-by-if-statement-if-bits0x20-0x20
		# Reshape
		# Each image consists of 784 pixels
		# 	=> 28 pixels in x-axis and 28 pixels in y-axis 
		X_and_y[i] = fetch(file)[0x10:].reshape((-1, 784))
	else:
		X_and_y.append(i)
		# Reducing the dataset from 60.008/10.008 to flat 60.000/10.000
		# The first lines are irrelevant due to the documentation from Lecun
		X_and_y[i] = fetch(file)[8:]

# Generate four variables
X_train, y_train, X_test, y_test = X_and_y